<a href="https://colab.research.google.com/github/Kinbloom/Moringa-Bluecars/blob/master/Copy_of_Moringa_Data_Science_Prep_W4_Independent_Project_2019_07_Mary_Kinya_Python_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Load the Dataset

Import the necessary libraries.

In [0]:
import pandas as pd
import numpy as np

In [0]:
car_service_dataset = pd.read_csv('http://bit.ly/Autolibdataset')
car_service_dataset.head()

,Unnamed: 0,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
0,0,24 Rue Vulpian,3,3,1,0,1,operational,Paris,NaN,paris-vulpian-24,STATION,"48.8310493, 2.3460584",75013,Paris/Vulpian/24,operational,NaN,1,station,ok,nonexistent,2018,4,1,0,0
1,1,8 Boulevard Richard Lenoir,0,0,0,0,1,operational,Paris,NaN,paris-richardlenoir-8,STATION,"48.8545026, 2.370183",75011,Paris/Richard Lenoir/8,operational,NaN,3,station,ok,nonexistent,2018,4,1,0,0
2,2,4 Rue Charles Bossut,3,3,0,0,1,operational,Paris,NaN,paris-charlesbossut-4,STATION,"48.8431847, 2.3837959",75012,Paris/Charles Bossut/4,operational,NaN,2,station,ok,nonexistent,2018,4,1,0,0
3,3,3 avenue Claude Vellefaux,4,4,0,0,0,operational,Paris,NaN,paris-claudevellefaux-3,STATION,"48.8740623, 2.3703034",75010,Paris/Claude Vellefaux/3,operational,NaN,1,station,ok,nonexistent,2018,4,1,0,0
4,4,236 Boulevard Voltaire,1,1,0,0,1,operational,Paris,NaN,paris-voltaire-236,STATION,"48.852023, 2.389719",75011,Paris/Voltaire/236,operational,NaN,4,station,ok,nonexistent,2018,4,1,0,0


In [0]:
#Convert the dataset to a csv file to avoid consantly reloading the dataset.

car_service_csv = car_service_dataset.to_csv('car_service.csv',encoding='utf-8',index=False)

In [0]:
#Load the dataframe from the csv file and proceed from there.

car_service_dataset = pd.read_csv('car_service.csv')
car_service_dataset.head()

,Unnamed: 0,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
0,0,24 Rue Vulpian,3,3,1,0,1,operational,Paris,NaN,paris-vulpian-24,STATION,"48.8310493, 2.3460584",75013,Paris/Vulpian/24,operational,NaN,1,station,ok,nonexistent,2018,4,1,0,0
1,1,8 Boulevard Richard Lenoir,0,0,0,0,1,operational,Paris,NaN,paris-richardlenoir-8,STATION,"48.8545026, 2.370183",75011,Paris/Richard Lenoir/8,operational,NaN,3,station,ok,nonexistent,2018,4,1,0,0
2,2,4 Rue Charles Bossut,3,3,0,0,1,operational,Paris,NaN,paris-charlesbossut-4,STATION,"48.8431847, 2.3837959",75012,Paris/Charles Bossut/4,operational,NaN,2,station,ok,nonexistent,2018,4,1,0,0
3,3,3 avenue Claude Vellefaux,4,4,0,0,0,operational,Paris,NaN,paris-claudevellefaux-3,STATION,"48.8740623, 2.3703034",75010,Paris/Claude Vellefaux/3,operational,NaN,1,station,ok,nonexistent,2018,4,1,0,0
4,4,236 Boulevard Voltaire,1,1,0,0,1,operational,Paris,NaN,paris-voltaire-236,STATION,"48.852023, 2.389719",75011,Paris/Voltaire/236,operational,NaN,4,station,ok,nonexistent,2018,4,1,0,0


### Cleaning the dataset

What columns do I need for my analysis process?
*   Unnamed: 0
*   Cars (Since it is basically a duplicate of the bluecars counter, it can be dropped)
*   Charge Slots
*  Charging Status
*  Displayed comment
*  Geo point
*  Scheduled at
*  ID
*  Rental 
*  Slots
*  year
*  month
*  minute



In [0]:
car_service = car_service_dataset.drop(['Unnamed: 0','Charge Slots','Charging Status','Rental status','Slots','year','month','minute','Cars','Displayed comment','Geo point','Scheduled at','ID'],axis=1)

In [0]:
car_service.head()

,Address,Bluecar counter,Utilib counter,Utilib 1.4 counter,City,Kind,Postal code,Public name,Station type,Status,Subscription status,day,hour
0,24 Rue Vulpian,3,1,0,Paris,STATION,75013,Paris/Vulpian/24,station,ok,nonexistent,1,0
1,8 Boulevard Richard Lenoir,0,0,0,Paris,STATION,75011,Paris/Richard Lenoir/8,station,ok,nonexistent,1,0
2,4 Rue Charles Bossut,3,0,0,Paris,STATION,75012,Paris/Charles Bossut/4,station,ok,nonexistent,1,0
3,3 avenue Claude Vellefaux,4,0,0,Paris,STATION,75010,Paris/Claude Vellefaux/3,station,ok,nonexistent,1,0
4,236 Boulevard Voltaire,1,0,0,Paris,STATION,75011,Paris/Voltaire/236,station,ok,nonexistent,1,0


Quick analysis of the dataset

In [0]:
car_service.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14421829 entries, 0 to 14421828
Data columns (total 13 columns):
Address                object
Bluecar counter        int64
Utilib counter         int64
Utilib 1.4 counter     int64
City                   object
Kind                   object
Postal code            int64
Public name            object
Station type           object
Status                 object
Subscription status    object
day                    int64
hour                   int64
dtypes: int64(6), object(7)
memory usage: 1.4+ GB


In [0]:
car_service.shape

(14421829, 20)

Testing for accuracy involves double - checking whether the Cars column matches the Bluecars counter column

In [0]:
car_service_dataset[~(car_service_dataset.Cars == car_service_dataset['Bluecar counter'])]

,Unnamed: 0,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute


Lack of values shows redundancy hence it was right to drop that column.

Identify duplicate rows and drop them thereafter.

In [0]:
car_service[car_service.duplicated()].head()

,Address,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Kind,Postal code,Public name,Rental status,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
1119,43 Rue de Monceau,0,0,0,2,operational,Paris,STATION,75008,Paris/Monceau/43,operational,6,station,ok,nonexistent,2018,4,1,0,0
1120,330 rue des PyrÃ©nÃ©es,2,0,0,1,operational,Paris,SPACE,75020,Paris/PyrÃ©nÃ©es/330,operational,3,full_station,ok,operational,2018,4,1,0,0
1121,28 Avenue de la Porte de Villiers,2,0,0,1,operational,Paris,STATION,75017,Paris/Porte de Villiers/28,operational,4,station,ok,nonexistent,2018,4,1,0,0
1122,97 Rue de Charenton,1,0,0,0,operational,Paris,STATION,75012,Paris/Charenton/97,operational,2,station,ok,nonexistent,2018,4,1,0,0
1123,17 boulevard Saint-Martin,2,0,0,0,nonexistent,Paris,STATION,75003,Paris/Saint-Martin/17,operational,1,station,ok,nonexistent,2018,4,1,0,0


In [0]:
car_service = car_service.drop_duplicates()

In [0]:
car_service.shape

(14278939, 20)

Checking for null values

In [0]:
# Checking for null values

car_service.isnull().sum()

Address                0
Bluecar counter        0
Utilib counter         0
Utilib 1.4 counter     0
City                   0
Kind                   0
Postal code            0
Public name            0
Station type           0
Status                 0
Subscription status    0
day                    0
hour                   0
dtype: int64

Changing the case of column names to lower, stripping whitespaces, removing periods and replacing spaces with underscore

In [0]:
car_service.columns = car_service.columns.str.lower().str.strip().str.replace(' ', '_').str.replace('.', '')

car_service.columns

Index(['address', 'bluecar_counter', 'utilib_counter', 'utilib_14_counter',
       'city', 'kind', 'postal_code', 'public_name', 'station_type', 'status',
       'subscription_status', 'day', 'hour'],
      dtype='object')

Adding a use column for the three car counters (bluecar_counter, utilib_counter and utilib_14_counter)

+ve number means that a car was returned
-ve number means that a car was picked
0 means that nothing happened, i.e no car was picked nor returned

In [0]:
#Creating column for bluecar usage
car_service['bluecar_use'] = car_service.bluecar_counter.diff()

#Creating column for utilib usage
car_service['utilib_use'] = car_service.utilib_counter.diff()

#Creating column for utilib 14 usage
car_service['utilib_14_use'] = car_service.utilib_14_counter.diff()

In [0]:
car_service.head()

,address,bluecar_counter,utilib_counter,utilib_14_counter,city,kind,postal_code,public_name,station_type,status,subscription_status,day,hour,bluecar_use,utilib_use,utilib_14_use
0,24 Rue Vulpian,3,1,0,Paris,STATION,75013,Paris/Vulpian/24,station,ok,nonexistent,1,0,NaN,NaN,NaN
1,8 Boulevard Richard Lenoir,0,0,0,Paris,STATION,75011,Paris/Richard Lenoir/8,station,ok,nonexistent,1,0,-3.0,-1.0,0.0
2,4 Rue Charles Bossut,3,0,0,Paris,STATION,75012,Paris/Charles Bossut/4,station,ok,nonexistent,1,0,3.0,0.0,0.0
3,3 avenue Claude Vellefaux,4,0,0,Paris,STATION,75010,Paris/Claude Vellefaux/3,station,ok,nonexistent,1,0,1.0,0.0,0.0
4,236 Boulevard Voltaire,1,0,0,Paris,STATION,75011,Paris/Voltaire/236,station,ok,nonexistent,1,0,-3.0,0.0,0.0


# Analysis Questions

## Bluecars Analysis

#### What is the most popular hour of the day for picking up a shared electric car in the city of Paris over the month of April 2018 ? (Specifically Bluecars)

In [0]:
car_service.bluecar_use.unique()

array([nan, -3.,  3.,  1.,  0., -1.,  4., -2., -5.,  2.,  5.,  6., -4.,
       -6., -7.,  7.])

In [0]:
car_service[(car_service.city == 'Paris') & (car_service.bluecar_use == -7.0)].groupby('hour')['hour'].count().sort_values(ascending = False).head(1)

hour
13    15
Name: hour, dtype: int64

The most popular hour of the day for picking cars is when the difference is at maximum (-7) is 1300 hours (Basically 1p.m.)

#### What is the most popular hour for returning cars?

In [0]:
car_service[car_service.bluecar_use == 7.].groupby('hour')['hour'].count().sort_values(ascending = False).head(1)

hour
4    37
Name: hour, dtype: int64

The most popular hour for returning cars is when the difference is at the highest possible positive difference (7.0) is 0400 hours (Basically 4a.m.)

#### What station is the most popular overall ?

In [0]:
car_service[(car_service.kind == 'STATION') & (car_service.status == 'ok')].groupby('public_name')[['kind']].count().sort_values(by = 'kind', ascending  = False).head(1)

,kind
public_name,
Boulogne-Billancourt/Jean JaurÃ¨s/245,606


The most popular station overall is Boulogne-Billancourt /Jean JaurA s/245

#### What is the most popular station at the most popular hour ?

In [0]:
car_service[(car_service.kind == 'STATION') & (car_service.hour == 13)  & (car_service.status == 'ok')].groupby('public_name').count()[['kind']].sort_values(by = 'kind', ascending = False).head(1)

,kind
public_name,
Paris/Exelmans/32,31


The most popular station at the most popular picking hour is Paris/Exelmans/32

#### What postal code is most popular when picking up Bluecars ?

In [0]:
car_service[car_service.bluecar_use < 0].groupby('postal_code').count()[['bluecar_use']].sort_values(by = 'bluecar_use', ascending = False).head(1)

,bluecar_use
postal_code,
75015,9816


#### What postal code is the most popular for picking up Bluecars at the most popular picking hour?

In [0]:
car_service[(car_service.bluecar_use < 0) & (car_service.hour == 13)].groupby('postal_code').count()[['bluecar_use']].sort_values(by = 'bluecar_use', ascending = False).head(1)

,bluecar_use
postal_code,
75015,473


#### Does the most popular station belong to that postal code overall ?

In [0]:
car_service[car_service.public_name == 'Boulogne-Billancourt/Jean JaurÃ¨s/245'][['public_name', 'postal_code']].head(1)

,public_name,postal_code
522,Boulogne-Billancourt/Jean JaurÃ¨s/245,92100


The most popular station does not belong to that postal code.

#### Does the most popular station belong to that postal code at the most popular picking hour?

In [0]:
car_service[(car_service.public_name == 'Paris/Exelmans/32') & (car_service.hour == 14)][['public_name', 'postal_code']].head(1)

,public_name,postal_code
949524,Paris/Exelmans/32,75016


The most popular station does not belong to the postal code at the most popular picking hour.

## Utilib Analysis 

#### Using Utilib instead of  Bluecars

In [0]:
car_service.utilib_use.unique()

array([nan, -1.,  0.,  1.,  2., -2.,  3., -3.,  4., -4.])

This is the range of the differences logged in the Utilib_use column.
The maximum negative number (-4) shows when the picking of the cars was highest.
Similly, the maximum positive number (4) shows when the returning of the cars was highest.

#### What is the most popular hour of the day for picking up a shared electric car in the city of Paris over the month of April 2018 ? (Specifically Utilib)

In [0]:
car_service[(car_service.city == 'Paris') & (car_service.utilib_use.min() < 0 )].groupby('hour')['hour'].count().sort_values(ascending = False).head(1)

hour
19    13200
Name: hour, dtype: int64

#### What is the most popular hour for returning cars?

In [0]:
car_service[car_service.utilib_use > 0 ].groupby('hour')['hour'].count().sort_values(ascending = False).head(1)

hour
19    1303
Name: hour, dtype: int64

#### What station is the most popular overall ?

In [0]:
car_service[(car_service.kind == 'STATION') & (car_service.status == 'ok')].groupby('public_name')[['kind']].count().sort_values(by = 'kind', ascending  = False).head(1)

,kind
public_name,
Boulogne-Billancourt/Jean JaurÃ¨s/245,606


#### What is the most popular station at the most popular hour ?

In [0]:
car_service[(car_service.kind == 'STATION') & (car_service.hour == 19)  & (car_service.status == 'ok')].groupby('public_name').count()[['kind']].sort_values(by = 'kind', ascending = False).head(1)

,kind
public_name,
Boulogne-Billancourt/Jean JaurÃ¨s/245,42


#### What postal code is most popular when picking up Bluecars ?

In [0]:
car_service[car_service.utilib_use < 0].groupby('postal_code').count()[['utilib_use']].sort_values(by = 'utilib_use', ascending = False).head(1)

,utilib_use
postal_code,
75015,1521


#### What postal code is the most popular for picking up Bluecars at the most popular picking hour?

In [0]:
car_service[(car_service.utilib_use < 0) & (car_service.hour == 19)].groupby('postal_code').count()[['utilib_use']].sort_values(by = 'utilib_use', ascending = False).head(1)

,utilib_use
postal_code,
75015,92


#### Does the most popular station belong to that postal code overall ?

In [0]:
car_service[car_service.public_name == 'Boulogne-Billancourt/Jean JaurÃ¨s/245'][['public_name', 'postal_code']].head(1)

,public_name,postal_code
522,Boulogne-Billancourt/Jean JaurÃ¨s/245,92100


#### Does the most popular station belong to that postal code at the most popular picking hour?

In [0]:
car_service[(car_service.public_name == 'Paris/Exelmans/32') & (car_service.hour == 14)][['public_name', 'postal_code']].head(1)

,public_name,postal_code
949524,Paris/Exelmans/32,75016


## Utilib 1.4

#### Using Utilib 1.4  instead of  Bluecars

In [0]:
car_service.utilib_14_use.unique()

array([nan,  0.,  1., -1.,  2., -2., -3.,  3.,  4., -4.])

This is the range of the differences logged in the Utilib_use column.
The maximum negative number (-4) shows when the picking of the cars was highest.
Similly, the maximum positive number (0) shows when the returning of the cars was highest.

#### What is the most popular hour of the day for picking up a shared electric car in the city of Paris over the month of April 2018 ? (Specifically Utilib 1.4)

In [0]:
car_service[(car_service.city == 'Paris') & (car_service.utilib_14_use.min() < 0 )].groupby('hour')['hour'].count().sort_values(ascending = False).head(1)

hour
19    13200
Name: hour, dtype: int64

#### What is the most popular hour for returning cars?

In [0]:
car_service[car_service.utilib_14_use > 0 ].groupby('hour')['hour'].count().sort_values(ascending = False).head(1)

hour
20    2377
Name: hour, dtype: int64

#### What station is the most popular overall ?

In [0]:
car_service[(car_service.kind == 'STATION') & (car_service.status == 'ok')].groupby('public_name')[['kind']].count().sort_values(by = 'kind', ascending  = False).head(1)

,kind
public_name,
Boulogne-Billancourt/Jean JaurÃ¨s/245,606


#### What is the most popular station at the most popular hour ?

In [0]:
car_service[(car_service.kind == 'STATION') & (car_service.hour == 19)  & (car_service.status == 'ok')].groupby('public_name').count()[['kind']].sort_values(by = 'kind', ascending = False).head(1)

,kind
public_name,
Boulogne-Billancourt/Jean JaurÃ¨s/245,42


#### What postal code is most popular when picking up Utilib cars ?

In [0]:
car_service[car_service.utilib_use < 0].groupby('postal_code').count()[['utilib_use']].sort_values(by = 'utilib_use', ascending = False).head(1)

,utilib_use
postal_code,
75015,1521


#### What postal code is the most popular for picking up Utilib cars at the most popular picking hour?

In [0]:
car_service[(car_service.utilib_use < 0) & (car_service.hour == 19)].groupby('postal_code').count()[['utilib_use']].sort_values(by = 'utilib_use', ascending = False).head(1)

,utilib_use
postal_code,
75015,92


#### Does the most popular station belong to that postal code overall ?

In [0]:
car_service[car_service.public_name == 'Boulogne-Billancourt/Jean JaurÃ¨s/245'][['public_name', 'postal_code']].head(1)

,public_name,postal_code
522,Boulogne-Billancourt/Jean JaurÃ¨s/245,92100


#### Does the most popular station belong to that postal code at the most popular picking hour?

In [0]:
car_service[(car_service.public_name == 'Paris/Exelmans/32') & (car_service.hour == 19)][['public_name', 'postal_code']].head(1)

,public_name,postal_code
1288463,Paris/Exelmans/32,75016
